## Numerical Analysis - Fall semester 2024

# Serie 10 - Linear systems

Package imports.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

<hr style="clear:both">

### Convergence of Jacobi's method (From the exam of 21/01/2014)

Given
$$
A =
\begin{pmatrix}
  1  & \beta \\
  -2 & 1
\end{pmatrix}
$$

and $\mathbf{b} \in \mathbb{R}^{2}$ which form the linear system $A\mathbf{x}=\mathbf{b}$.

<div class="alert alert-success">
    
**Exercise 1 (Theoretical):** For which values of $\beta$ does Jacobi's method converge for any $\mathbf{b}$ and any initial vector $\mathbf{x}^{(0)}$? 

1. For $\beta \in (-1, 0]$

2. For $\beta = -1/2$

3. For $\beta \in [0,1)$

4. For $\beta \in (-1/2,1/2)$

5. For $\beta \in (-1/2,\infty)$
</div>


<hr style="clear:both">

### Gradient method and generalizations

<div class="alert alert-success">
    
**Exercise 2:** Complete the function `gradient_method` which implements the gradient method (Algorithm 5.3 in lecture notes) for iteratively solving the linear system $A \mathbf{x} = \mathbf{b}$.
</div>

In [ ]:
def gradient_method(A, b, x_0, n_max, tol):
    x = x_0
    r = b - A @ x
    k = 0
    while np.linalg.norm(r) > tol * np.linalg.norm(b) and k < n_max:
        # YOUR CODE HERE
        raise NotImplementedError()
    return x, r, k

We now want to compare the gradient method with the Jacobi method from last week. Therefore, we provide you with an implementation of the Jacobi method.

In [ ]:
def jacobi(A, b, x_0, n_max, tol):
    P = np.diag(np.diag(A))
    x = x_0
    r = b - A @ x
    k = 0
    while np.linalg.norm(r) > tol * np.linalg.norm(b) and k < n_max:
        z = np.linalg.solve(P, r)
        x = x + z
        r = b - A @ x
        k = k + 1
    return x, r, k

As a test case, we consider the symmetric positive definite Laplacian matrix $A \in \mathbb{R}^{m \times m}$ you are already familiar with from two weeks ago. You can generate it with the following function:

In [ ]:
def laplacian_matrix(m):
    n = int(np.sqrt(m))
    d = np.ones(n ** 2)
    mat = sp.sparse.spdiags([- d, 2 * d, - d], [-1, 0, 1], n, n)
    I = sp.sparse.eye(n)
    A = sp.sparse.kron(I, mat) + sp.sparse.kron(mat, I) 
    return A.toarray()

A = laplacian_matrix(100)
plt.title(r"$100 \times 100$ Laplacian matrix")
plt.spy(A)
plt.show()

<div class="alert alert-success">

**Exercise 3:** Solve the system $A \mathbf{x} = \mathbf{b}$ for the Laplacian matrix $A \in \mathbb{R}^{100 \times 100}$ from Exercise 2 for number of iterations $n_{\max} = 10, 20, 30, \dots, 200$ with the Jacobi and the gradient method. We "handpick" the solution $\mathbf{x} = (\frac{1}{m}, \frac{2}{m}, \dots, \frac{m}{m})^{\top}$, and set $\mathbf{b} = A \mathbf{x}$. Use $\mathbf{x}^{(0)} = \boldsymbol{0}$ and take `tol = 0` such that the stopping criterion won't be satisfied. Plot the errors $\lVert \mathbf{x}_c - \mathbf{x} \rVert$ of the iterative solution $\mathbf{x}_c$ for both methods against the number of iterations $n_{\max}$ for a logarithmic $y$-axis (`plt.semilogy`). Explain what you observe.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<div class="alert alert-success">

**Exercise 4:** Repeat the previous exercise to compare the gradient method with the conjugate gradient method. Explain the difference by analyzing the condition number of $A$ using `np.linalg.cond(A)`.

*Hint:* You can use `x, _ = sp.sparse.linalg.cg(A, b, x_0, maxiter=n_max, rtol=tol)` to obtain the conjugate gradient solution `x` of the linear system with matrix `A` and right-hand side `b`, with starting vector `x_0`, maximum number of iterations `n_max`, and tolerance `tol`.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

### Preconditioned conjugate gradient

The below function generates the matrix $A \in \mathbb{R}^{m \times m}$, such that the diagonal entries are $a_{jj} = 0.5 + \sqrt{j}, j=1, 2, \dots, m$, and the first and $\sqrt{m}-th$ sub- and superdiagonal are $-1$. This matrix has quite a high condition number, meaning, it is *ill-conditioned*.

In [ ]:
def ill_conditioned_matrix(m):
    n = int(np.sqrt(m))
    e = np.ones(n ** 2)
    v = np.sqrt(np.arange(n ** 2))
    A = sp.sparse.spdiags([-e, -e, 0.5*e + v, -e, -e], [-n, -1, 0, 1, n])
    return A.toarray()

A = ill_conditioned_matrix(100)
plt.title(r"$100 \times 100$ ill-conditioned matrix")
plt.spy(A)
plt.show()

print("condition number: κ(A) = {:.3f}".format(np.linalg.cond(A)))

Since the speed with which the conjugate gradient method convergences depends on the condition number $\kappa(A)$, we can use a matrix $M$ such that the new condition number $\tilde{\kappa}(M A)$ is smaller, i.e. $M$ is a good approximation to the inverse of $A$. To measure how fast the conjugate gradient method converges for different choices of preconditioning $M$, we can use the following function `precond_conjugate_gradient_niter`, which takes as inputs a linear system with matrix `A`, a right-hand side `b`, and a preconditioner $M$, and uses the preconditioned conjugate gradient method with starting vector `x_0`, maximum number of iterations `n_max`, and tolerance `tol` to solve the system. The function returns the number of iterations it took to find a solution which satisfies the stopping criterion with tolerance `tol`.

In [ ]:
def precond_conjugate_gradient_niter(A, b, M, x_0, n_max, tol):
    global niter
    niter = 0
    def counter_conjugate(arr):
        global niter
        niter = niter + 1
    sp.sparse.linalg.cg(A, b, x_0, maxiter=n_max, rtol=tol, M=M, callback=counter_conjugate)
    return niter

We now want to study the dependence of the number of iterations it takes the algorithm to converge for different choices of preconditioning. 

* No preconditioning:
$$
M = I_{m}
$$
where $I_{m}$ is the $m \times m$ identity matrix.

* Jacobi preconditioning:
$$
M = \operatorname{diag}(A)^{-1},
$$
i.e. the matrix with the diagonal of $A$, and all other entries being zero.

* Tridiagonal preconditioning:
$$
M = \operatorname{tridiag}(A)^{-1},
$$
i.e. the matrix which just contains the diagonal and both off-diagonals of $A$, and all other entries being zero.

<div class="alert alert-warning">
    
**Warning:** The NumPy function `np.diag` called on an $m \times m$ matrix $A$ returns its diagonal as a length `m` vector, and not the matrix $\operatorname{diag}(A)$ which is everywhere zero except for the diagonal entries, which coincide with the ones in $A$. However, `np.diag` called on a length $m$ vector will generate an $m \times m$ diagonal matrix with this vector on its diagonal. Hence, you can generate $\operatorname{diag}(A)$ by calling the `np.diag` function twice, i.e. `A_diag = np.diag(np.diag(A))`.
</div>

<div class="alert alert-success">

**Exercise 5:** For the system $A \mathbf{x} = \boldsymbol{1}$ with $A \in \mathbb{R}^{100 \times 100}$ generated with `ill_conditioned_matrix` and $\boldsymbol{1} \in \mathbb{R}^{100}$ the vector of ones, use the function `precond_conjugate_gradient_niter` to determine how many iterations the preconditioned conjugate gradient method requires to satisfy the stopping criterion with tolerance `tol = 1e-16` for the different choices of the preconditioner $M$ mentioned above. Use a random normal vector $\mathbf{x}^{(0)}$ (generated with `np.random.randn(m)`) as starting vector and choose `n_max` sufficiently large to not be reached before the stopping criterion is satisfied. Can you explain the difference?

*Hint:* For a matrix $A$, you can extract the sub-diagonal with `np.diag(A, k=-1)` and super-diagonal with `np.diag(A, k=1)`. Same goes for generating a sub-diagonal and super-diagonal matrix from a vector.
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

<hr style="clear:both">

## The end

Easy! You have already finished the tenth exercise notebook.